In [20]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K
import toml
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_yaml
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D
from numpy import array
from numpy import hstack
from math import floor

from ipynb.fs.full.V07_Utils  import *

In [21]:
######################################################################
# Load Configuration
######################################################################

config_path = 'C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Code/V07_config.toml'
cfg = toml.load(config_path)

######################################################################
# Load Directories 
######################################################################

# Predicted Traces
Path_dataset01_test = cfg['Path_dataset01']

# GRU quat
Path_results_dataset01_GRU_quat = cfg['Path_results_dataset01'] + "GRU_quat/"

# GRU pos
Path_results_dataset01_GRU_pos = cfg['Path_results_dataset01'] + "GRU_pos/"

# Baseline quat
Path_results_dataset01_Baseline_quat = cfg['Path_results_dataset01'] + "Baseline_quat/"

# Baseline pos
Path_results_dataset01_Baseline_pos = cfg['Path_results_dataset01'] + "Baseline_pos/"

# Tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  0


In [22]:
def compute_pos_metrics(true_pos, preds_pos):
        # Compute Eucliden and angular distances
        euc_dists = np.linalg.norm(true_pos - preds_pos, axis=1)
        #print(euc_dists)
        # Mean Absolute Error (MAE)
        mae_euc = sum(euc_dists) / euc_dists.shape[0]
        # Root Mean Squared Error (RMSE)
        rmse_euc = np.sqrt((euc_dists ** 2).mean())
        return euc_dists, mae_euc, rmse_euc


def compute_quat_metrics(true_rot, preds_rot,true_rot_quat, preds_rot_quat):
        # Compute Eucliden and angular distances
        ang_dists = np.array([Quaternion.distance(q1, q2) for q1, q2 in zip(true_rot_quat, preds_rot_quat)]) #(w, x, y, z)
        euc_dists = np.linalg.norm(true_rot - preds_rot, axis=1)
        # Mean Absolute Error (MAE)
        #mae_ang = np.rad2deg(np.sum(ang_dists) / ang_dists.shape[0])
        #mae_euc = np.rad2deg(np.sum(euc_dists) / euc_dists.shape[0])
        
        mae_ang = (np.sum(ang_dists) / ang_dists.shape[0])
        mae_euc = (np.sum(euc_dists) / euc_dists.shape[0])
        
        # Root Mean Squared Error (RMSE)
        rmse_ang = np.rad2deg(np.sqrt((ang_dists ** 2).mean()))
        rmse_euc = np.rad2deg(np.sqrt((euc_dists ** 2).mean()))
        return euc_dists, ang_dists, mae_ang, mae_euc, rmse_ang, rmse_euc

def eval_quat(true_rot, preds_rot):
        true_rot_quat = np.array([Quaternion(q) for q in true_rot]) #(w, x, y, z)
        preds_rot_quat = np.array([Quaternion(q) for q in preds_rot]) #(w, x, y, z)    
        return compute_quat_metrics(true_rot, preds_rot,true_rot_quat, preds_rot_quat)
    
def compute_euler_metrics(Dataset_path_result_euler, keyword):
        df = pd.read_csv(Dataset_path_result_euler)
        df_pred = np.deg2rad(df[keyword + '_pred']%360)          
        df_true = np.deg2rad(df[keyword + '_true']%360)
        ang_dists = []
        for i in range(len(df_true)):
                ang_dists1 = np.abs(df_true[i] - df_pred[i])
                ang_dists2 = np.abs(2*np.pi + df_true[i] - df_pred[i])
                ang_dists3 = np.abs(-2*np.pi + df_true[i] - df_pred[i])
                ang_dists_single = min(ang_dists1,ang_dists2,ang_dists3)
                ang_dists_single = np.rad2deg(ang_dists_single)
                ang_dists.append(ang_dists_single)
        ang_dists_arr = np.array(ang_dists)
        # Mean Absolute Error (MAE)
        mae_ang = (sum(ang_dists_arr) / ang_dists_arr.shape[0])
        return ang_dists_arr, mae_ang
    
    
def compute_euler_metrics_radiant(Dataset_path_result_euler, keyword):
        df = pd.read_csv(Dataset_path_result_euler)
        df_pred = (df[keyword + '_pred'])          
        df_true = (df[keyword + '_true'])
        ang_dists = []
        for i in range(len(df_true)):
                sin = np.abs(df_true[i] - df_pred[i])
                cos = df_true[i] - df_pred[i]
                ang_dists_single = np.arctan2(sin,cos)
                ang_dists_single = np.rad2deg(ang_dists_single)
                ang_dists.append(ang_dists_single)
        ang_dists_arr = np.array(ang_dists)
        # Mean Absolute Error (MAE)
        mae_ang = (sum(ang_dists_arr) / ang_dists_arr.shape[0])
        return ang_dists_arr, mae_ang

In [23]:
############################
# Evaluation quaternion 
############################

def eval_quaternion(Path_dataset, Path_result):
    
    for csv in get_csv_files(Path_dataset, ""):
            print(csv)
            trace_id = os.path.splitext(os.path.basename(csv))[0]
            csv_df = load_df(csv)
            csv_df = csv_df[:-1]
            in_seq1 = csv_df[['qw_pred']].to_numpy()
            in_seq2 = csv_df[['qx_pred']].to_numpy()
            in_seq3 = csv_df[['qy_pred']].to_numpy()
            in_seq4 = csv_df[['qz_pred']].to_numpy()

            in_seq5 = csv_df[['qw_true']].to_numpy()
            in_seq6 = csv_df[['qx_true']].to_numpy()
            in_seq7 = csv_df[['qy_true']].to_numpy()
            in_seq8 = csv_df[['qz_true']].to_numpy()


            #print("pred len is" + str(len(csv_df))
            dataset_pred = hstack((in_seq1, in_seq2, in_seq3, in_seq4))
            dataset_true = hstack((in_seq5, in_seq6, in_seq7, in_seq8))
            dataset_pred = array(dataset_pred)
            dataset_true = array(dataset_true)
            
            print(len(dataset_true))

            euc_dists, ang_dists, mae_ang, mae_euc, rmse_ang, rmse_euc = eval_quat(dataset_pred, dataset_true)
            print('Number of dists: ' + str(ang_dists.shape[0]))
            #print('Number of euclidean dists: ' + str(euc_dists.shape[0]))
            print('MAE_ang rotation is : ' + str(mae_ang))
            #print('RMSE_ang rotation is : ' + str(rmse_ang))
            print('MAE_euc rotation is : ' + str(mae_euc))
            #print('RMSE_euc rotation is : ' + str(rmse_euc))
            print("----")

            ############ Save metrices
            result_one_experiment = list([[mae_ang,mae_euc,rmse_ang,rmse_euc]])
            df_results = pd.DataFrame(result_one_experiment, columns=['mae_ang','mae_euc','rmse_ang','rmse_euc'])
            df_results.to_csv(os.path.join(Path_result, trace_id + '_evaluation.csv'), index=False)

            ############ Save angular distances
            dist_one_experiment = list(zip(ang_dists,euc_dists)) 
            dists_one_experiment = pd.DataFrame(dist_one_experiment, columns=['ang_dists','euc_dists'])
            dists_one_experiment.to_csv(os.path.join(Path_result, trace_id + '_dists.csv'), index=False)


In [24]:
############################
# Evaluation position 
############################

def eval_position(Path_dataset, Path_result):
    
    for csv in get_csv_files(Path_dataset, "cont"):
            print(csv)
            trace_id = os.path.splitext(os.path.basename(csv))[0]
            csv_df = load_df(csv)
            csv_df = csv_df[:-1]
            in_seq1 = csv_df[['x_pred']].to_numpy()
            in_seq2 = csv_df[['y_pred']].to_numpy()
            in_seq3 = csv_df[['z_pred']].to_numpy()
            
            in_seq5 = csv_df[['x_true']].to_numpy()
            in_seq6 = csv_df[['y_true']].to_numpy()
            in_seq7 = csv_df[['z_true']].to_numpy()
            


            #print("pred len is" + str(len(csv_df))
            dataset_pred = hstack((in_seq1, in_seq2, in_seq3))
            dataset_true = hstack((in_seq5, in_seq6, in_seq7))
            dataset_pred = array(dataset_pred)
            dataset_true = array(dataset_true)



            euc_dists, mae_euc, rmse_euc = compute_pos_metrics(dataset_pred, dataset_true)
            print('Number of dists: ' + str(euc_dists.shape[0]))
            print('MAE_euc position is : ' + str(mae_euc))
            print('RMSE_euc rotation is : ' + str(rmse_euc))
            print("----")

            ############ Save metrices
            result_one_experiment = list([[mae_euc,rmse_euc]])
            df_results = pd.DataFrame(result_one_experiment, columns=['mae_euc','rmse_euc'])
            df_results.to_csv(os.path.join(Path_result, trace_id + '_evaluation.csv'), index=False)

            ############ Save angular distances
            dist_one_experiment = np.array(euc_dists).flatten()
            dists_one_experiment = pd.DataFrame(dist_one_experiment, columns=['euc_dists'])
            dists_one_experiment.to_csv(os.path.join(Path_result, trace_id + '_dists.csv'), index=False)


In [25]:
############################
# Evaluation quaternion 
############################

def eval_quaternion_from_keras(Path_dataset, Path_result):
    
    for csv in get_csv_files(Path_dataset, "y_pred"):
            print(csv)
            trace_id = os.path.splitext(os.path.basename(csv))[0]
            csv_df_pred = load_df(csv)
            csv_df_pred = csv_df_pred[:-1]
            in_seq1 = csv_df_pred[['qw_pred']].to_numpy()
            in_seq2 = csv_df_pred[['qx_pred']].to_numpy()
            in_seq3 = csv_df_pred[['qy_pred']].to_numpy()
            in_seq4 = csv_df_pred[['qz_pred']].to_numpy()
            
            dataset_pred = hstack((in_seq1, in_seq2, in_seq3, in_seq4))
            dataset_pred = array(dataset_pred)
            
    for csv in get_csv_files(Path_dataset, "y_true"):
            print(csv)
            trace_id = os.path.splitext(os.path.basename(csv))[0]
            csv_df_true = load_df(csv)
            csv_df_true = csv_df_true[:-1]

            in_seq5 = csv_df_true[['qw_true']].to_numpy()
            in_seq6 = csv_df_true[['qx_true']].to_numpy()
            in_seq7 = csv_df_true[['qy_true']].to_numpy()
            in_seq8 = csv_df_true[['qz_true']].to_numpy()


            #print("pred len is" + str(len(csv_df))
            
            dataset_true = hstack((in_seq5, in_seq6, in_seq7, in_seq8))
            dataset_true = array(dataset_true)



            euc_dists, ang_dists, mae_ang, mae_euc, rmse_ang, rmse_euc = eval_quat(dataset_pred, dataset_true)
            print('Number of dists: ' + str(ang_dists.shape[0]))
            #print('Number of euclidean dists: ' + str(euc_dists.shape[0]))
            print('MAE_ang rotation is : ' + str(mae_ang))
            #print('RMSE_ang rotation is : ' + str(rmse_ang))
            print('MAE_euc rotation is : ' + str(mae_euc))
            #print('RMSE_euc rotation is : ' + str(rmse_euc))
            print("----")

            ############ Save metrices
            result_one_experiment = list([[mae_ang,mae_euc,rmse_ang,rmse_euc]])
            df_results = pd.DataFrame(result_one_experiment, columns=['mae_ang','mae_euc','rmse_ang','rmse_euc'])
            df_results.to_csv(os.path.join(Path_result, trace_id + '_evaluation.csv'), index=False)

            ############ Save angular distances
            dist_one_experiment = list([[ang_dists,euc_dists]])
            dists_one_experiment = pd.DataFrame(dist_one_experiment, columns=['ang_dists','euc_dists'])
            dists_one_experiment.to_csv(os.path.join(Path_result, trace_id + '_dists.csv'), index=False)


# GRU quat

In [26]:
###################################################
# Provide Model ID
model_spec = 'Delta_GRU_v01_Neurons_32__DR_0.5__LR_0.01__Batch_128__Epochs_20' 
model_ID = cfg['6_1_model_ID']
###################################################

# Load Traces
Path_results_dataset01_GRU_quat_model_id_traces =  Path_results_dataset01_GRU_quat + "/" + model_ID + "/Traces/" 

# Result Directory 
Path_results_dataset01_GRU_quat_model_id_evaluation =  Path_results_dataset01_GRU_quat + "/" + model_ID + "/Evaluation/" 

In [27]:
# Evaluation dataset 01
eval_quaternion(Path_results_dataset01_GRU_quat_model_id_traces,
               Path_results_dataset01_GRU_quat_model_id_evaluation)

C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_quat//7_2409_v01/Traces\1_full_continious_result_rot.csv
34959
Number of dists: 34959
MAE_ang rotation is : 0.004425328956229867
MAE_euc rotation is : 0.00442501158561548
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_quat//7_2409_v01/Traces\2_full_continious_result_rot.csv
14959
Number of dists: 14959
MAE_ang rotation is : 0.004175325396031246
MAE_euc rotation is : 0.004175311057506365
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_quat//7_2409_v01/Traces\3_full_continious_result_rot.csv
44959
Number of dists: 44959
MAE_ang rotation is : 0.004288661237580069
MAE_euc rotation is : 0.004287874406546111
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_quat//7_2409_v01/Traces\4_full_continious_result_rot.csv
69959
Number of dists: 69959
MAE_ang rotation is : 0.004429900015646611
M

# GRU pos

In [17]:
###################################################
# Provide Model ID
#model_spec = 'GRU_v01_Neurons_4__DR_0__LR_1000__Batch_512__Epochs_2000' 
model_ID = cfg['7_1_model_ID']
###################################################

# Load Traces
Path_results_dataset01_GRU_pos_model_id_traces =  Path_results_dataset01_GRU_pos + "/" + model_ID + "/Traces/" 

# Result Directory 
Path_results_dataset01_GRU_pos_model_id_evaluation =  Path_results_dataset01_GRU_pos + "/" + model_ID + "/Evaluation/" 

In [18]:
Path_results_dataset01_GRU_pos_model_id_traces

'C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_pos//7_2409_v01/Traces/'

In [19]:
# Evaluation dataset 01
eval_position(Path_results_dataset01_GRU_pos_model_id_traces,
               Path_results_dataset01_GRU_pos_model_id_evaluation)

C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_pos//7_2409_v01/Traces\1_full_continious_result_delta_pos.csv
Number of dists: 34959
MAE_euc position is : 0.0014082335087031359
RMSE_euc rotation is : 0.0021194035262601277
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_pos//7_2409_v01/Traces\2_full_continious_result_delta_pos.csv
Number of dists: 14959
MAE_euc position is : 0.001549704824078638
RMSE_euc rotation is : 0.0021980652831266674
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_pos//7_2409_v01/Traces\2_full_continious_result_pos_abs.csv
Number of dists: 14964
MAE_euc position is : 0.009577273347824182
RMSE_euc rotation is : 0.011506064939214258
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/GRU_pos//7_2409_v01/Traces\3_full_continious_result_delta_pos.csv
Number of dists: 44959
MAE_euc position is : 0.0014111001980977112

# Baseline quat

In [33]:
# Load Traces
Path_results_dataset01_Baseline_quat_model_id_traces =  Path_results_dataset01_Baseline_quat + "/" + "Traces/" 

# Result Directory 
Path_results_dataset01_Baseline_quat_model_id_evaluation =  Path_results_dataset01_Baseline_quat + "/" + "Evaluation/" 

In [34]:
# Evaluation dataset 01
eval_quaternion(Path_results_dataset01_Baseline_quat_model_id_traces,
               Path_results_dataset01_Baseline_quat_model_id_evaluation)

C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_quat//Traces\1_full_continious_result_delta_rot.csv
34959
Number of dists: 34959
MAE_ang rotation is : 0.030995005385561478
MAE_euc rotation is : 0.030984082869161097
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_quat//Traces\2_full_continious_result_delta_rot.csv
14959
Number of dists: 14959
MAE_ang rotation is : 0.028337943424258542
MAE_euc rotation is : 0.028331630875108104
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_quat//Traces\3_full_continious_result_delta_rot.csv
44959
Number of dists: 44959
MAE_ang rotation is : 0.034926083535348755
MAE_euc rotation is : 0.034914606346651535
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_quat//Traces\4_full_continious_result_delta_rot.csv
69959
Number of dists: 69959
MAE_ang rotation is : 0.03247080452513652
M

# Baseline pos 

In [35]:
# Load Traces
Path_results_dataset01_Baseline_pos_model_id_traces =  Path_results_dataset01_Baseline_pos + "/" + "Traces/" 

# Result Directory 
Path_results_dataset01_Baseline_pos_model_id_evaluation =  Path_results_dataset01_Baseline_pos + "/" + "Evaluation/" 

In [36]:
# Evaluation dataset 01
eval_position(Path_results_dataset01_Baseline_pos_model_id_traces,
               Path_results_dataset01_Baseline_pos_model_id_evaluation)

C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_pos//Traces\1_full_continious_result_pos.csv
Number of dists: 34959
MAE_euc position is : 0.013094881851583792
RMSE_euc rotation is : 0.018522650418232096
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_pos//Traces\2_full_continious_result_pos.csv
Number of dists: 14959
MAE_euc position is : 0.0129708775729299
RMSE_euc rotation is : 0.017120930714351643
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_pos//Traces\3_full_continious_result_pos.csv
Number of dists: 44959
MAE_euc position is : 0.015474101224156371
RMSE_euc rotation is : 0.020980013202343738
----
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Results/01_Dataset/Baseline_pos//Traces\4_full_continious_result_pos.csv
Number of dists: 69959
MAE_euc position is : 0.014667790246804686
RMSE_euc rotation is : 0.02217956515836492
----
C:/